<a href="https://colab.research.google.com/github/Md-Jahid-Hasan/Data-Science-AI-ML/blob/master/Hand_Number_Digits_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

if torch.cuda.is_available():
    device = torch.device("cuda:0")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
batch_size = 25
learning_rate = 0.001
num_epochs = 50

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
class CollectData(Dataset):
    def __init__(self, x):
        self.X = x
        
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        label = torch.tensor(self.X[index][1])
        img = transform(self.X[index][0])
        return (img, label)

In [ ]:
training_data = np.load('gdrive/My Drive/Colab Notebooks/Dataset/handNumber_train_data.npy', allow_pickle=True)
test_data = np.load('gdrive/My Drive/Colab Notebooks/Dataset/handNumber_test_data.npy', allow_pickle=True)

In [ ]:
test_data.shape

(3600, 2)

In [ ]:
train_dataset = CollectData(training_data)
test_dataset = CollectData(test_data)

In [ ]:
len(train_dataset)


18000

In [ ]:
len(test_dataset)

3600

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=True)

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding='same')
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding='same')
        self.conv3 = nn.Conv2d(128, 256, 3, padding='same')
        self.fc1 = nn.Linear(256*4*4, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 12)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 256*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (image, labels) in enumerate(train_loader):
        output = model(image.to(device)).to(device)
        # print(labels)
        # break
        loss = criterion(output, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch} Loss: {loss}')

Epoch: 0 Loss: 2.4734673500061035
Epoch: 1 Loss: 2.4915812015533447
Epoch: 2 Loss: 2.487149715423584
Epoch: 3 Loss: 2.480733871459961
Epoch: 4 Loss: 2.491262674331665
Epoch: 5 Loss: 2.475123643875122
Epoch: 6 Loss: 2.4701101779937744
Epoch: 7 Loss: 2.476919412612915
Epoch: 8 Loss: 2.4805376529693604
Epoch: 9 Loss: 2.476950168609619
Epoch: 10 Loss: 2.4841675758361816
Epoch: 11 Loss: 2.465271472930908
Epoch: 12 Loss: 2.4706974029541016
Epoch: 13 Loss: 2.4707868099212646
Epoch: 14 Loss: 2.4633140563964844
Epoch: 15 Loss: 2.459681749343872
Epoch: 16 Loss: 2.454498291015625
Epoch: 17 Loss: 2.4324965476989746
Epoch: 18 Loss: 2.4153859615325928
Epoch: 19 Loss: 2.349759101867676
Epoch: 20 Loss: 2.2640154361724854
Epoch: 21 Loss: 2.020455837249756
Epoch: 22 Loss: 1.6456999778747559
Epoch: 23 Loss: 1.223951816558838
Epoch: 24 Loss: 0.8475494384765625
Epoch: 25 Loss: 0.6783384680747986
Epoch: 26 Loss: 0.3632281422615051
Epoch: 27 Loss: 0.30076807737350464
Epoch: 28 Loss: 0.18381881713867188
Epoch

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(12)]
    n_class_samples = [0 for i in range(12)]

    for image, labels in test_loader:
        labels = labels.to(device)
        output = model(image.to(device)).to(device)
        _, predicted = torch.max(output, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        print(n_samples, n_correct)
        break

25 2


In [ ]:
conv1 = nn.Conv2d(3, 64, 3, padding='same')
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(64, 128, 3, padding='same')
conv3 = nn.Conv2d(128, 256, 3, padding='same')

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
images.shape

torch.Size([25, 3, 32, 32])

In [ ]:
x = conv1(images)
print(x.shape)
x = pool(x)
print(x.shape)
x = conv2(x)
print(x.shape)
x = pool(x)
print(x.shape)
x = conv3(x)
print(x.shape)
x = pool(x)
print(x.shape)

torch.Size([25, 64, 32, 32])
torch.Size([25, 64, 16, 16])
torch.Size([25, 128, 16, 16])
torch.Size([25, 128, 8, 8])
torch.Size([25, 256, 8, 8])
torch.Size([25, 256, 4, 4])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1


[0, 0]